In [14]:
#import dependency
import pandas as pd
import os
import requests
import numpy as np
from config import g_key
import gmaps
import gmaps.datasets
#import depedency to export gmap as html
from ipywidgets.embed import embed_minimal_html

In [15]:
#load the city vacation data file
file_to_load = os.path.join("weather_data", "WeatherPy_vacation.csv")
city_vacation_data_df = pd.read_csv(file_to_load)
city_vacation_data_df.head(5)

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Port Lincoln,AU,64.60,clear sky,-34.73,135.87,Port Lincoln Hotel
1,8,Ushuaia,AR,69.80,scattered clouds,-54.80,-68.30,Albatros Hotel
2,14,Bluff,NZ,55.99,light rain,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
3,26,Cape Town,ZA,66.99,few clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
4,31,Busselton,AU,66.99,clear sky,-33.65,115.33,Observatory Guest House


In [16]:
#filter the city vacation data with the cities we are interested in
# lets plan for a vacation to Catamarca(Argentina) and Coquimbo,Constitucion,Talcahuano (Chile)
city_1_df = city_vacation_data_df[(city_vacation_data_df['City'] == 'Catamarca')]
city_2_df = city_vacation_data_df[(city_vacation_data_df['City'] == 'Coquimbo')]
city_3_df = city_vacation_data_df[(city_vacation_data_df['City'] == 'Constitucion')]
city_4_df = city_vacation_data_df[(city_vacation_data_df['City'] == 'Talcahuano')]

In [17]:
#configure gmaps with the api key
gmaps.configure(api_key=g_key)

In [18]:
#create latitude and longitude for all four destinations
city_1_Lat_Lng = list(city_1_df[["Lat", "Lng"]].itertuples(index=False, name=None))[0]
city_2_Lat_Lng = list(city_2_df[["Lat", "Lng"]].itertuples(index=False, name=None))[0]
city_3_Lat_Lng = list(city_3_df[["Lat", "Lng"]].itertuples(index=False, name=None))[0]
city_4_Lat_Lng = list(city_4_df[["Lat", "Lng"]].itertuples(index=False, name=None))[0]

In [29]:
#create gmaps driving layer from Catamarca to Talcahuano via Coquimbo and Constitucion
fig = gmaps.figure()
Catamarca_Talcahuano_Directions = gmaps.directions_layer(city_1_Lat_Lng, city_4_Lat_Lng,
                                                        waypoints=[city_2_Lat_Lng,city_3_Lat_Lng],
                                                        travel_mode="DRIVING")
fig.add_layer(Catamarca_Talcahuano_Directions)

#export the driving directions as html
embed_minimal_html('images/WeatherPy_travel_map.html', views=[fig])

#show fig
fig


Figure(layout=FigureLayout(height='420px'))

In [32]:
#combining all dataframes
itenary_cities_data = [city_1_df, city_2_df, city_3_df, city_4_df]
itenary_cities_data_df = pd.concat(itenary_cities_data)
itenary_cities_data_df.head()



,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
52,245,Catamarca,AR,60.01,overcast clouds,-28.47,-65.79,Amerian Catamarca Park Hotel
45,229,Coquimbo,CL,68.00,clear sky,-29.95,-71.34,Hotel San Juan
58,270,Constitucion,CL,62.53,clear sky,-35.33,-72.42,Hotel Boutique Las Azucenas
32,170,Talcahuano,CL,68.00,few clouds,-36.72,-73.12,Hotel Terramar


In [33]:
itenary_cities_locations = itenary_cities_data_df[["Lat","Lng"]]

#create infoblock template whose values will be replace iterating
info_block_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}\xb0F</dd>
</dl>
"""
#create infoblock for display
hotel_info = [info_block_template.format(**row) for index, row in itenary_cities_data_df.iterrows()]

itenary_cities_marker_layer = gmaps.marker_layer(itenary_cities_locations, info_box_content=hotel_info)



In [37]:
#create gmaps driving layer from Catamarca to Talcahuano via Coquimbo and Constitucion
fig = gmaps.figure()

#fig.add_layer(Catamarca_Talcahuano_Directions)
fig.add_layer(itenary_cities_marker_layer)

#export the driving directions as html
embed_minimal_html('images/WeatherPy_travel_map_markers.html', views=[fig])

#show fig
fig

Figure(layout=FigureLayout(height='420px'))